# The heat method for distance computation

In this notebook we implement the heat method shown in (Crane, Weischedel and Wardetzky, 2017) for triangle meshes.

In [1]:
import pyvista as pv
pv.set_jupyter_backend('static')

import numpy as np
from scipy.sparse import lil_matrix, identity
import scipy.sparse.linalg as spla

import matplotlib.pyplot as plt

In [2]:
MAX_FACES = 15_000

#  Mesh from https://www.thingiverse.com/thing:6703649/files
mesh = pv.PolyData('resources/3D_models/blue_whale/files/ballena_azul_Lowpoly.stl')

num_faces = len(mesh.faces) // 4

if num_faces > MAX_FACES:
    mesh = mesh.decimate(1.0 - (MAX_FACES / num_faces))
    print('Decimated mesh, from ', num_faces, 'faces to', len(mesh.faces)//4, 'faces')

nodes = mesh.points
triangles = np.delete(mesh.faces.reshape(-1, 4), 0, 1)
normals = mesh.compute_normals()['Normals']

avg_h = 0.0
for face in triangles:
    x,y,z = nodes[face]
    avg_h += sum([np.linalg.norm(y-x), np.linalg.norm(z-x), np.linalg.norm(y-z)])

avg_h = avg_h / (3*len(triangles))

print('# Nodes  :', nodes.shape)
print('# Faces  :', triangles.shape)
print('# Normals:', normals.shape) 
print('# avgh   :', avg_h)

normals_field = pv.vector_poly_data([np.mean(nodes[T], axis=0) for T in triangles], normals)

def save(filename, mesh, **kwargs):
    pl = pv.Plotter()
    pl.add_mesh(mesh, **kwargs)
    pl.save_graphic(filename)
    return pl

# Nodes  : (1845, 3)
# Faces  : (3686, 3)
# Normals: (3686, 3)
# avgh   : 3.472326


In [3]:
save('images/heat_method_mesh.svg', mesh);
save('images/heat_method_normals.svg', normals_field.glyph(orient='vectors', scale='mag'));
#  mesh.plot()

<img src="images/heat_method_mesh.svg"></img>

<img src="images/heat_method_normals.svg"></img>

In [4]:
from mesh_operators import laplace_matrix

In [5]:
L, M = laplace_matrix(nodes, triangles)

## Steady State Heat Equation

Let $\Omega$ be a compact closed orientable surface in $\mathbb{R}^3$. Suppose that heat source $f$ is acting on $\Omega$. The heat $u(t,x)$ at time $t$ at $x\in\Omega$ can be describe with the following equation:
$$
    \dot u(t,x) - \Delta u(t,x) = f(t, x) \qquad t > 0
$$
where an initial condition $u(0,x)$ must be given. This is the heat equation and its steady states corresponds to $\dot u(t,x) = 0$.

In [6]:
default_cmap = plt.get_cmap('seismic')

source_indices = [np.argmax(nodes[:, 0]), np.argmax(nodes[:, 0] + nodes[:, 1])]
source_field = np.zeros(len(nodes))
source_field[source_indices] = [(-1)**i for i in range(len(source_indices))]

vmax = np.max(source_field)
vmin = np.min(source_field)

save('images/heat_method_sources.svg', mesh, scalars=source_field, clim=[vmin, vmax], cmap=default_cmap);
#  mesh.plot(scalars=source_field, clim=[vmin, vmax], cmap=default_cmap)

<img src="images/heat_method_sources.svg"></img>

In [7]:
steady_heat = -spla.spsolve(L.tocsr(), M @ source_field)

In [8]:
save('images/heat_method_steady_heat.svg', mesh, scalars=steady_heat, cmap=default_cmap);
#  mesh.plot(scalars=steady_heat, cmap=default_cmap)

<img src="images/heat_method_steady_heat.svg"></img>

# Heat Equation and Geodesics

Let $\Gamma$ be a closed subset of $\Omega$.. We are interested in the geodesic field generated by $\Gamma$. That is, we seek for a function $\phi\colon\Omega\to\mathbb{R}$ such that $\phi(x) = \text{distance}(x,\Gamma)$ where the distance is computed over the surface $\Omega$. The nonlinear pde describing $\phi$ reads as follow:
$$
\left\{
\begin{aligned}
    |\nabla \phi| &= 1 &&, \text{ on } \Omega\setminus\Gamma \\
    \phi &= 0 &&, \text{ over } \Gamma
\end{aligned}
\right.
$$
And it's known as the **eikonal equational**. Nonlinear problems are not easy to solve. The heat method proposes an approach inspired by physics: heat flow first along paths of maximum conductivity i.e. the shortest paths. Let's think of $\Gamma$ as a heat source, then the very first seconds of dissipitation will encode the shortest paths of any point to $\Gamma$. This was observation was made by field medalist [Srinavasa Varadhan](https://en.wikipedia.org/wiki/S._R._Srinivasa_Varadhan). The relationship between heat flow and geodesics, which is obtained by solving for the distance in the formula for the fundamental solution $h_t$ of the heat equation, reads:
$$
    d(x,y) = \sqrt{-4 \lim_{t\to 0} t \log h_t(x,y)}
    \tag{V}
$$
where $h_t$ solves the solves:
$$
\left\{
\begin{aligned}
    \dot{h}_{t}(x,\cdot) &= \Delta h_{t}(x,\cdot) &&, \qquad t> 0 \\
    \lim_{t\to 0} h_t(x,\cdot) &= \delta_x(\cdot)
\end{aligned}
\right.
\tag{F}
$$
So $h_t(x,\cdot)$ gives us the heat distribution at time $t$ when inducing unit heat from a point source $x\in\Omega$. The objective now is to solve the heat equation for short times and build the distance from there.

The limit in Varadhan relationship is not looking good from a discretization point of view. What is really useful about this relation
is that the distance field and the heat kernel are somewhat proportional. Meaning that both fields (for small $t$) have the same directional flow. So we will actually solve for the heat equation and perform a posteriori adjustment in order to obtain an approximation of the geodesic field.

Let $X(\cdot)$ be the normalize flow of the heat kernel $h_t$, that we know from Varadhan's relationships, is equal to the geodesic flow. We'll look for an approximation of the geodesic field $\phi$ such that minimizes the energy
$$
    \frac{1}{2} \int (\nabla \phi - X)^2,
$$
which yields the Euler-Lagrange equations $\Delta \phi = \nabla\cdot X$. Once found $\phi$, we need to perform a correction such that $\phi$ vanishes on $\Gamma$.

In order to solve (F), we discretized $\Delta$ using the discrete cotan Laplacian and backward Euler step for time ($\dot{h}_t \approx (h_t - h_0)/t$). Then,
$$
    \dot h_t (x,\cdot) = \Delta h_t(x,\cdot)
    \approx
    h_t - h_0 = tL h_t(x,\cdot)
    \iff
    (I-tL) h_t = h_0,
$$
where $h_0 = \delta_x$ is a point source. In the same paper, Varadhan's extends the relationship~(V) to hold for sets like $\Gamma$.
Therefore, we end up with:
$$
    (I-tL) u_t = \delta_{\Gamma}.
$$

**The heat method algorithm**:
1. Compute the solution $u_t$ for an appropiate small $t$.
2. Normalized the direction field. That is, set $X = -\nabla u_t / \lvert{\nabla u_t}\rvert$.
3. Solve the poisson equation $\Delta \phi = \nabla \cdot X$.

In [9]:
def geodesic_field(
    nodes: list, triangles: list, normals: list,
    Gamma: list, 
    L,
    t: float) -> list:
    '''
    Input:
        - nodes : list of 3-vectors. Represent the nodes in the mesh.
        - triangles : list of 3-list. Each 3-list contains the indices in nodes that conform
        that triangle.
        - Gamma : list of nodes (indices). For a point x, the distance is computed between
                x and Gamma.
    Ouput:
        - The list of nodes with the value of the distance field.
    '''
    from mesh_operators import div, grad_matrix
    # 1. Compute heat flow
    rhs = np.zeros(len(nodes))
    rhs[Gamma] = 1.0
    
    u = spla.spsolve((identity(len(nodes)) - t*L.tocsr()), rhs)
    save('images/heat_method_ut.svg', mesh);
    #  mesh.plot(scalars=u, text=r'$u_t$ for small $t$')
    
    # 2. Normalized flow
    grad_mat = grad_matrix(nodes, triangles, normals)
    grad_u = np.array([mat @ u[T] for mat,T in  zip(grad_mat, triangles)])
    X = -np.array([v/np.linalg.norm(v) for v in grad_u])
    
    grad_field = pv.vector_poly_data([np.mean(nodes[T],axis=0) for T in triangles], [x/np.linalg.norm(x) for x in X])
    #  grad_field.glyph(orient='vectors', scale='mag').plot(text='X=-normalized(grad u_t)')
    save('images/heat_method_grad_field.svg', grad_field.glyph(orient='vectors', scale='mag')); 
    
    divX = div(nodes, triangles, normals, X)
    #  mesh.plot(scalars=divX, text='Divergence of X')
    save('images/heat_method_div_field.svg', mesh, scalars=divX);
    
    # 3. Solve Poisson equation to get the actual distances (this came from the minimization problem)
    phi = spla.spsolve(L.tocsr(), divX)
    return phi-np.min(phi)

In [10]:
dist = geodesic_field(nodes, triangles, normals, source_indices, L, t=avg_h**2) 

In [11]:
geod_cmap = plt.get_cmap('prism_r')
save('images/heat_method_geodesic.svg', mesh, scalars=dist, cmap=geod_cmap);
#  mesh.plot(scalars=dist, cmap=geod_cmap)

<img src="images/heat_method_geodesic.svg"></img>

# References
* Crane, K., Weischedel, C. and Wardetzky, M., 2017. The heat method for distance computation. Communications of the ACM, 60(11), pp.90-99. Available at: https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/ (Accessed: 9 May 2025)